# 1. 라이브러리 불러오기

In [6]:
import pandas as pd
import numpy as np

# 2. 데이터 불러오기

In [7]:
df = pd.read_csv('../data/carrier_reefer_data_01_to_08_and_12.csv')

In [8]:
df.head()

,alarm_codes,ambient_temperature,made_cd,operating_mode,operating_mode_str,reefer_id,return_air_temperature,supply_air_temperature,temperature_setpoint,when_created
0,NaN,-8.29,CARRIER,6.0,IDLE,RDMU5216890,2.92,3.09,3.0,2022-01-01 00:00:23
1,"[""AL60""]",3.90,CARRIER,3.0,COOL,RJCU6995820,-19.83,-20.47,-20.0,2022-01-01 00:01:08
2,NaN,-1.06,CARRIER,6.0,IDLE,CTEU5865614,-24.43,-25.05,-24.0,2022-01-01 00:01:18
3,"[""dAL86""]",9.62,CARRIER,3.0,COOL,SDCU6045746,-17.57,-19.11,-20.0,2022-01-01 00:01:26
4,NaN,-6.06,CARRIER,3.0,COOL,UKFU6720496,-28.10,-30.61,-30.0,2022-01-01 00:01:32


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883975 entries, 0 to 883974
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   alarm_codes             165319 non-null  object 
 1   ambient_temperature     881033 non-null  float64
 2   made_cd                 883975 non-null  object 
 3   operating_mode          879638 non-null  float64
 4   operating_mode_str      879638 non-null  object 
 5   reefer_id               883975 non-null  object 
 6   return_air_temperature  883975 non-null  float64
 7   supply_air_temperature  883849 non-null  float64
 8   temperature_setpoint    883975 non-null  float64
 9   when_created            883975 non-null  object 
dtypes: float64(5), object(5)
memory usage: 67.4+ MB


In [10]:
# 문자열을 datetime으로 변경
import datetime

df['when_created'] = df['when_created'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

# 3. DEFROST가 발생한 데이터 찾기

## 3-1. 설정온도가 변하지 않는 컨테이너 찾기

In [16]:
print(f"CARRIER 컨테이너의 전체 개수는 {df['reefer_id'].nunique()}개 입니다.")

CARRIER 컨테이너의 전체 개수는 102개 입니다.


In [17]:
# 1. 컨테이너ID별로 분리

# 2. 설정 온도가 냉동이면서 설정 온도가 변하지 않는 경우만 필터링

In [31]:
import math

reefer_id_list = df['reefer_id'].unique()
temperature_list = []
for reefer_id in reefer_id_list:
    temperature_setpoint = df.loc[df['reefer_id'] == reefer_id, 'temperature_setpoint'].unique()
    if len(temperature_setpoint) == 1 and temperature_setpoint[0] < 0:
        temperature_list.append(math.floor(temperature_setpoint[0]))

pd.Series(temperature_list).value_counts()

-20    11
-18     4
-25     2
-19     2
-3      1
-21     1
-17     1
dtype: int64

- 영하 20도인 데이터가 가장 많으므로 영하 20도인 데이터만 사용한다.

In [36]:
import math

reefer_id_list = df['reefer_id'].unique()
reefer_below_20 = []
for reefer_id in reefer_id_list:
    temperature_setpoint = df.loc[df['reefer_id'] == reefer_id, 'temperature_setpoint'].unique()
    if len(temperature_setpoint) == 1 and math.floor(temperature_setpoint[0]) == -20.0:
        print(reefer_id, df.loc[df['reefer_id'] == reefer_id].shape[0])
        reefer_below_20.append(reefer_id)

RJCU6995820 31087
BMOU9003717 21743
CBHU2805710 47
SKRU9200955 17
GESU9370000 22
SKRU8300707 2795
CRXU5278311 7
CWCU5722568 1766
CCLU1038527 5
TRIU6681373 2524
IKRU2301291 1671


## 3-2. 찾은 컨테이너에서 DEFROST가 발생한 데이터 찾기

In [47]:
reefer_in_defrost = []

for reefer_id in reefer_below_20:
    operating_mode_list = list(df.loc[df['reefer_id'] == reefer_id, 'operating_mode_str'].unique())
    if "DEFROST" in operating_mode_list:
        print(f"{reefer_id}컨테이너의 DEFROST 발생횟수: {df.loc[df['reefer_id'] == reefer_id, 'operating_mode_str'].value_counts()['DEFROST']}번")
        reefer_in_defrost.append(reefer_id)

RJCU6995820컨테이너의 DEFROST 발생횟수: 5303번
BMOU9003717컨테이너의 DEFROST 발생횟수: 920번
CBHU2805710컨테이너의 DEFROST 발생횟수: 1번
SKRU9200955컨테이너의 DEFROST 발생횟수: 1번
GESU9370000컨테이너의 DEFROST 발생횟수: 5번
SKRU8300707컨테이너의 DEFROST 발생횟수: 410번
CWCU5722568컨테이너의 DEFROST 발생횟수: 94번
TRIU6681373컨테이너의 DEFROST 발생횟수: 62번
IKRU2301291컨테이너의 DEFROST 발생횟수: 48번


## 3-3. DEFROST가 시작하기 전과 끝난 이후 각각 6개의 데이터가 있는 것만 추출하기

- DEFROST 전후 1시간 간격으로 데이터를 끊을 생각이다.

In [48]:
reefer_in_defrost

['RJCU6995820',
 'BMOU9003717',
 'CBHU2805710',
 'SKRU9200955',
 'GESU9370000',
 'SKRU8300707',
 'CWCU5722568',
 'TRIU6681373',
 'IKRU2301291']

In [49]:
def pop_sequence_number(num_list):
    """
    DEFROST가 발생한 인덱스를 뽑아내는 함수입니다.

    Inputs
    ----------
    num_list: DEFROST가 발생한 인덱스 번호(1차원 리스트)

    Return
    ------
    packet: DEFROST가 발생한 인덱스 번호 묶음(2차원 리스트)
    """
    packet = []
    tmp = []

    v = num_list.pop(0)
    tmp.append(v)

    while (len(num_list) > 0):
        vv = num_list.pop(0)
        if v+1 == vv:
            tmp.append(vv)
            v = vv
        else:
            packet.append(tmp)
            tmp = []
            tmp.append(vv)
            v = vv
    
    packet.append(tmp)
    return packet

In [65]:
defrost_idx_per_reefer = {}

# 컨테이너별 DEFROST가 발생한 인덱스 추출
for reefer_id in reefer_in_defrost:
    aa = df.loc[(df['reefer_id'] == reefer_id)].reset_index(drop=True)
    defrost_idx_per_reefer[reefer_id] = list(aa.loc[aa['operating_mode_str'] == "DEFROST"].index) 

In [66]:
idx = pop_sequence_number(defrost_idx_per_reefer["RJCU6995820"])

In [69]:
aa = df.loc[(df['reefer_id'] == "RJCU6995820")].reset_index(drop=True)
aa.iloc[idx[0][0]:idx[0][-1]]

,alarm_codes,ambient_temperature,made_cd,operating_mode,operating_mode_str,reefer_id,return_air_temperature,supply_air_temperature,temperature_setpoint,when_created
123,"[""AL60""]",6.27,CARRIER,5.0,DEFROST,RJCU6995820,-19.15,-19.35,-20.0,2022-01-01 21:05:44
124,"[""AL60""]",4.84,CARRIER,5.0,DEFROST,RJCU6995820,-16.95,-17.68,-20.0,2022-01-01 21:16:16
125,"[""AL60""]",3.10,CARRIER,5.0,DEFROST,RJCU6995820,-12.24,-16.73,-20.0,2022-01-01 21:26:42
126,"[""AL60""]",1.91,CARRIER,5.0,DEFROST,RJCU6995820,-8.33,-15.94,-20.0,2022-01-01 21:36:57
127,"[""AL60""]",1.20,CARRIER,5.0,DEFROST,RJCU6995820,-5.66,-15.42,-20.0,2022-01-01 21:47:04
128,"[""AL60""]",0.70,CARRIER,5.0,DEFROST,RJCU6995820,-4.23,-15.08,-20.0,2022-01-01 21:57:24
129,"[""AL60""]",0.42,CARRIER,5.0,DEFROST,RJCU6995820,-3.13,-14.62,-20.0,2022-01-01 22:07:28
130,"[""AL60""]",0.17,CARRIER,5.0,DEFROST,RJCU6995820,-1.83,-14.25,-20.0,2022-01-01 22:17:38
131,"[""AL60""]",-0.14,CARRIER,5.0,DEFROST,RJCU6995820,-0.41,-13.83,-20.0,2022-01-01 22:28:00
132,"[""AL60""]",-0.41,CARRIER,5.0,DEFROST,RJCU6995820,0.82,-13.56,-20.0,2022-01-01 22:38:04


In [68]:
len(idx)

597

# 4. 시계열 데이터를 이미지로 변환하기

## 4-1. PAA로 추출한 시계열 데이터의 간격 맞추기

## 4-2. 시계열 데이터의 4가지 센서 데이터를 분할하고 전치시키기

## 4-3. GAFS, GAFD 방식을 각각 적용하여 이미지로 변환하기